# Search feature weighting

This notebook computes feature weights for a search ranking model.

As of early 2020, search ranking was being done by an ad-hoc pairwise comparison function that may not even be transitive. We want to replace it with a more structured and analyzable approach that can additional search features besides corpus frequency, such as cosine vector distance for now, with room for more features later.

The basic pieces are:
  - Use occurrence of a search result in the search survey as a 0-or-1 relevance variable
  - Create a relevance score from that using some fairly basic linear modelling techniques to compute best-fit feature weights
  - Measure success by the three10 score: the mean percentage of how many top-3 results from the linguist survey appear in the top 10 search results

There are many interesting possible future improvements here, such as:
  - Use occurence anywhere in sample, instead of top3 results only, for training
  - More precise training data, e.g., relevance rankings of 1-5
  - Handle homonyms in training data instead of matching purely on wordform text
  - More training data, specifically how many results per query we have human scores for
  - More features, e.g., tf-idf
  - Higher quality features, e.g., better stopword filtering in vector computations
  - Map features to have similar ranges and distributions to better allow the regression to more effectively compare them
  - Separate training and test sets
  - Fancier models
  - Better evaluation functions, such as discounted cumulative gain

That said, having all the pieces together, even in a very basic form, is already an improvement over the existing search, so let’s start with that.

## Preliminaries

Load some libraries. `weighting_nb_code.py` contains some more python-y code that was extracted from some exploratory jupyter notebooks once it was working ok.

In [1]:
import importlib


import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import weighting_nb_code

# Reload the code in `weighting_nb_code.py` by re-running this cell, or
# by copying the next line into other cells. If this reload mechanism
# proves insufficient, there is also `IPython.lib.deepreload`.
importlib.reload(weighting_nb_code);

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


First, if the JSON output file doesn’t already exist, we’ll run the `featuredump` management command to get our raw data. CVD search is not yet on by default, so we add a fancy query to enable it.

In [19]:
![ -f sample-features.json ] || \
    {weighting_nb_code.BASE_DIR.parent.parent}/crkeng-manage featuredump \
        --prefix-queries-with 'cvd:retrieval' \
        > sample-features.json

100%|█████████████████████████████████████████| 548/548 [00:39<00:00, 13.90it/s]


The loaded feature data looks like this:

In [20]:
data = weighting_nb_code.dataframe_from_featuredump('sample-features.json')
data

,relevance_score,target_language_affix_match,query,cosine_vector_distance,query_wordform_edit_distance,target_language_keyword_match,wordform_length,wordform_text,webapp_sort_rank,source_language_match,pos_match,analyzable_inflection_match,lemma_wordform_text,definitions,morpheme_ranking,source_language_keyword_match,is_lemma,source_language_affix_match
0,0.090367,True,about,0.000000,NaN,[about],4,ohci,1,None,0,None,ohci,"[[from there, thence, out of, CW], [with, by m...",None,[],True,None
1,0.089088,True,about,0.000000,NaN,[about],6,êtikwê,2,None,0,None,êtikwê,"[[apparently, I guess, I suppose, supposedly, ...",None,[],True,None
2,0.089088,True,about,0.000000,NaN,[about],6,êtokwê,3,None,0,None,êtokwê,"[[maybe, I guess, presumably, perhaps, CW], [a...",None,[],True,None
3,0.083974,True,about,0.000000,NaN,[about],14,êwako ôma ohci,4,None,0,None,êwako ôma ohci,"[[about this, CW], [this is why, this is the r...",None,[],True,None
4,0.066219,True,about,0.208314,NaN,[about],5,kêkât,5,None,0,None,kêkât,"[[just about, almost, CW]]",None,[],True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71964,-0.008841,None,they see us,NaN,NaN,[see],15,naspâtâpahcikêw,145,None,1,None,naspâtâpahcikêw,"[[s/he sees things the wrong way, s/he looks a...",None,[],True,None
71965,-0.009480,None,they see us,NaN,NaN,[see],16,kakwâyakâpamiwêw,146,None,0,None,kakwâyakâpamiwêw,"[[s/he sees people in a loathsome way, CW]]",None,[],True,None
71966,-0.009480,None,they see us,NaN,NaN,[see],16,kîhkâyâpamohiwêw,147,None,1,None,kîhkâyâpamohiwêw,"[[s/he makes people see (it/him) clearly, s/he...",None,[],True,None
71967,-0.010119,None,they see us,NaN,NaN,[see],17,kakwâyakâpahcikêw,148,None,0,None,kakwâyakâpahcikêw,"[[s/he sees things in a loathsome way, CW]]",None,[],True,None


Here’s the current combined result survey sample.

In [21]:
weighting_nb_code.survey()

,Query,Nêhiyawêwin 1,Nêhiyawêwin 2,Nêhiyawêwin 3
0,about,wayês,ohci,papâ
1,all,kahkiyaw,kapê,mâwaci
2,also,mîna,êkwa,kisik
3,and,êkwa,mîna,kisik
4,as,kisik,wiya,tâpiskôt
...,...,...,...,...
543,she sees him,wâpamêw,NaN,NaN
544,starblanket,atâhkakohp,acâhkosa kâ-otakohpit,NaN
545,star blanket,atâhkakohp,acâhkosa kâ-otakohpit,NaN
546,being taught,kiskinwahamâkosiw,NaN,NaN


And `weighting_nb_code.py` contains a function to annotate the `featuredump` results with the top3/three10 metrics.

In [22]:
weighting_nb_code.top3_and_310_stats(data, rank_column="webapp_sort_rank")[
    ["query", "wordform_text", "definitions", "actual_result_ranks", "top3", "three10"]
]

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,query,wordform_text,definitions,actual_result_ranks,top3,three10
0,"""horse""",misatim,"[[horse, CW]]","[3.0, 4.0]",100.0,100.0
1,'horse',misatim,"[[horse, CW]]","[5.0, 13.0]",100.0,50.0
2,Calgary,otôskwanihk,"[[Calgary, AB, CW]]",[1.0],100.0,100.0
3,Cree,nêhiyaw,"[[Nehiyaw, CW], [Cree, Cree man, First Nations...","[5.0, 26.0]",100.0,50.0
4,Cree language,nêhiyawêwin,"[[the Cree language, CW], [speaking Cree, CW],...",[1.0],100.0,100.0
...,...,...,...,...,...,...
543,yellow hat,osâwastotin,"[[yellow hat, CW]]",[1.0],100.0,100.0
544,you,kiya,"[[you, CW]]","[1.0, 2.0, 3.0]",100.0,100.0
545,young,oski,NaN,[],0.0,0.0
546,younger sibling,nisîmis,"[[my younger sibling (brother or sister), my y...",[6.0],100.0,100.0


## Initial results from dictionary code

Without any cosine-vector stuff, here are the current search stats we want to beat. 81.3% for top3, and 59.4% for three10.

In [23]:
import os
if os.path.isfile('sample-features-orig.json'):
    data_orig = weighting_nb_code.dataframe_from_featuredump('sample-features-orig.json')
    display(weighting_nb_code.top3_and_310_stats_summary(data_orig, rank_column="webapp_sort_rank"))

Note: this won’t exactly match what the django `/search-quality` pages report, because of some differences in determining exactly what the rank is. In the django code, if the results are `(non-lemma1, non-lemma2)`, we count the ranks as `(1, 3)` because the UI display of `non-lemma1` includes its lemma definition at rank 2. Here we skip that for now, but the results should be close enough.

And, for comparison, here are the stats when we added a very basic cosine vector distance model to the search:

In [24]:
weighting_nb_code.top3_and_310_stats_summary(data, rank_column="webapp_sort_rank")

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


top3       81.630170
three10    67.609489
dtype: float64

The top3 score—what percent of desired search results we see anywhere in the list—has gone up. That is, the vector model’s ability to resolve synonyms has improved recall. But the three10 score—what percent of desired search results are near the top—has gone down since we don’t have a good ranking mechanism.

## Modelling

At this point we have all the definition and feature data from the webapp loaded, and we could experiment by adding more data columns with additional features. Those features could be computed by Python code here, or loaded from data files.

For this first version, let’s stick with what we have:

In [25]:
def prep_results_for_regression(df):
    # The default value used for `fillna()` doesn’t matter if we
    # also have an indicator variable, but things get trickier
    # with logarithms.
    return df.assign(
        morpheme_ranking=df["morpheme_ranking"].fillna(1),
        has_morpheme_ranking=weighting_nb_code.has_col_as_int(df, "morpheme_ranking"),
        has_cosine_vector_distance=weighting_nb_code.has_col_as_int(df, "cosine_vector_distance"),
        cosine_vector_distance=df["cosine_vector_distance"].fillna(1.1),
        is_in_survey=df.apply(weighting_nb_code.is_in_survey, axis=1),
        keyword_match_len=df['target_language_keyword_match'].apply(len),
        pos_match=df["pos_match"].fillna(0),
    )

In [26]:
df = prep_results_for_regression(data)
results = smf.ols(
    """
    is_in_survey ~
        wordform_length
        + keyword_match_len
        + has_morpheme_ranking
        + morpheme_ranking
        + pos_match
        + np.log(1 + cosine_vector_distance)
    """,
    data=df,
).fit()
display(results.summary())
sorted_results = weighting_nb_code.rank_by_predictor(df, results)
weighting_nb_code.top3_and_310_stats_summary(sorted_results, rank_column="result_rank")

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           is_in_survey   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     1060.
Date:                Tue, 22 Mar 2022   Prob (F-statistic):               0.00
Time:                        13:03:58   Log-Likelihood:                 63750.
No. Observations:               71969   AIC:                        -1.275e+05
Df Residuals:                   71964   BIC:                        -1.274e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              0.0341      0.001     43.699      0.000       0.033       0.036
wordform_length                       -0.0021      0.000    -19.234      0.000      -0.002      -0.002
keyword_match_len                      0.0349      0.001     35.079      0.000       0.033       0.037
has_morpheme_ranking                -1.87e-17   3.03e-19    -61.778      0.000   -1.93e-17   -1.81e-17
morpheme_ranking                       0.0341      0.001     43.699      0.000       0.033       0.036
pos_match                              0.0010      0.002      0.496      0.620      -0.003       0.005
np.log(1 + cosine_vector_distance)    -0.1076      0.002    -57.838      0.000      -0.111      -0.104
==============================================================================
Omnibus:                   101870.978   Durbin-Watson:                   1.777
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19830622.157
Skew:                           8.738   Prob(JB):                         0.00
Kurtosis:                      82.421   Cond. No.                     1.50e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.33e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


top3       81.630170
three10    68.826034
dtype: float64

Objectively, we have increased how many of the top survey results we display at all from 81.3% to 83.4%, and we have increased the mean number of them that appear in the top 10 results per query from ~60% to ~70%. That’s great!

Let’s take a look at a sample query. Before:

In [27]:
(weighting_nb_code.top3_and_310_stats(data, rank_column='webapp_sort_rank')
     .query('query == "counts"'))[['query', 'actual_result_ranks', 'top3', 'three10']]

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,query,actual_result_ranks,top3,three10
126,counts,"[1.0, 2.0, 4.0]",100.0,100.0


When searching for ‘counts,’ before all the good results were showing up somewhere in the results, but none of them were in the top 10.

Now, with this new ranking model, the top results from the survey show up at the top of the search results, and even in 1, 2, 3 order:

In [28]:
(weighting_nb_code.top3_and_310_stats(sorted_results, rank_column='result_rank')
     .query('query == "counts"'))[['query', 'actual_result_ranks', 'top3', 'three10']]

/Users/jolenepoulin/.local/share/virtualenvs/morphodict-5QYDyuRz/lib/python3.9/site-packages/pandas/core/algorithms.py:794: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,query,actual_result_ranks,top3,three10
126,counts,"[1.0, 2.0, 4.0]",100.0,100.0


If we look in more detail at the results, we can see that the cosine vector distance and the morpheme ranking are being combined, but one doesn’t overrule the other. Rarer words generally appear later in the list, but a strong CVD score can move it earlier, and vice versa.

In [29]:
sorted_results.query("query == 'counts'").sort_values('score', ascending=False)[
    ['wordform_text', 'definitions', 'morpheme_ranking',
     'has_cosine_vector_distance',
     'cosine_vector_distance', 'is_in_survey', 'score']
 ].head(10)

,wordform_text,definitions,morpheme_ranking,has_cosine_vector_distance,cosine_vector_distance,is_in_survey,score
53159,akimêw,"[[s/he counts s.o., CW]]",1,1,5.960464e-08,1,0.090571
53161,akihtâw,"[[s/he counts s.t., CW]]",1,1,5.960464e-08,0,0.088505
53160,akihtam,"[[s/he counts s.t., CW], [s/he adds s.t., s/he...",1,1,5.960464e-08,1,0.088505
53162,akihcikêw,"[[s/he counts, CW]]",1,1,5.960464e-08,1,0.084373
53163,akihtâsow,"[[s/he counts, CW]]",1,1,5.960464e-08,0,0.084373
53164,wiyakimêw,"[[s/he counts s.o., CW], [s/he puts a price on...",1,1,5.960464e-08,0,0.084373
53165,wiyakihtam,"[[s/he counts s.t., CW], [s/he puts a price on...",1,1,5.960464e-08,0,0.082307
53166,akihtamawêw,"[[s/he counts (them) for s.o., CW]]",1,1,5.960464e-08,0,0.080241
53167,itakihcikêw,"[[s/he values things so, CW], [s/he fixes a pr...",1,1,5.960464e-08,0,0.080241
53168,itakimêw,"[[s/he counts s.o. thus, CW], [s/he value s.o....",1,1,1.710499e-01,0,0.069442


This is quite a bit better than only using the morpheme ranking:

In [30]:
if os.path.isfile('sample-features-orig.json'):
    display((data_orig.assign(is_in_survey=data_orig.apply(weighting_nb_code.is_in_survey, axis=1))
     .query("query == 'counts'").sort_values('webapp_sort_rank')[
        ['wordform_text', 'definitions', 'morpheme_ranking', 'is_in_survey']
     ]).head(10))

## Model export

While the model generated by the `statsmodels` library is `pickle`able, since it’s a fairly basic linear model, for now we will just print the parameters to use in the webapp.

In [31]:
print(results.params.to_json(indent=2))

{
  "Intercept":0.0340573975,
  "wordform_length":-0.0020658984,
  "keyword_match_len":0.0348514421,
  "has_morpheme_ranking":-1.869983896e-17,
  "morpheme_ranking":0.0340573975,
  "pos_match":0.0010077928,
  "np.log(1 + cosine_vector_distance)":-0.1076452857
}


And here are some test vectors for ensuring the implementation is working correctly.

In [35]:
import re

def print_test_vector(**kwargs):
    df = prep_results_for_regression(pd.DataFrame([{
        "query": "counts",
        "wordform_text": "",
        "target_language_keyword_match": [],
        "wordform_length": 0,
        "keyword_match_len": 0,
        "morpheme_ranking": np.nan,
        "cosine_vector_distance": np.nan,
        "pos_match": 0,
        **kwargs
    }]))
    ret = results.predict(df)[0]
    # future python feature “underscore as a decimal separator”
    # https://bugs.python.org/issue43624 would be handy here
    ret = f'{ret:_f}'
    if '.' in ret:
        l, r = ret.split('.')
        r = re.sub(r'(...)(?=.)', r'\1_', r)
        ret = f'{l}.{r}'
    print(ret)

In [36]:
print_test_vector()

-0.011_751


In [37]:
print_test_vector(cosine_vector_distance=0.7)

0.010_995


In [38]:
print_test_vector(morpheme_ranking=12.8)

0.390_126


In [39]:
print_test_vector(cosine_vector_distance=0.7, morpheme_ranking=12.8)

0.412_872


In [40]:
print_test_vector(cosine_vector_distance=0.7, morpheme_ranking=12.8, pos_match=5)

0.417_911
